### xESMF - problems occuring when creating a `xarray.Dataset` by using `xesmf.Regridder.__call__`

* `xesmf.Regridder.__call__` does not forward some coordinate variables to the regridded `xarray.Dataset` that are registered as `xarray.Dataset.data_vars` and not as `xarray.Dataset.coords` while it forwards others (like the input horizontal bounds) that should rather be replaced.
* Trying to use `xarray.open_dataset(path_to_ds, decode_coords='all')` registers all coordinate variables properly BUT removes significant metadata, not allowing `cf_xarray.accessor._get_item.drop_bounds` to distinguish between the coordinate variable and its bounds (fixed in >0.6.1, yet unreleased). 
* xarrays `decode_coords='all'` registers for example `ps` (surface pressure), that is required for a sigma hybrid vertical axis, under `xarray.Dataset.coords`, preventing `xESMF` from remapping it (it gets dropped).
* I am not aware of a `cf_xarray.CFaccessor.method` / `cf_xarray`-function that redefines / resets `xarray.Dataset.data_vars` and `xarray.Dataset.coords` of an `xarray.Dataset` in the desired manner. So I set up custom methods within `clisops.core.Grid`.

-> Which of the problems are considered problems of xESMF and therefore should be treated in xESMF (rather than by the user or other libraries)?

In [1]:
import numpy as np
import xarray as xr
import cf_xarray as cfxr
import xesmf as xe
import clisops as cl
from clisops.core import Grid
print("Using cf-xarray in version %s" % cfxr.__version__)
print("Using xESMF in version %s" % xe.__version__)
print("Using clisops in version %s" % cl.__version__)

from pathlib import Path
from git import Repo
import os

import warnings
warnings.simplefilter("ignore") 
#with warnings.catch_warnings():
#        warnings.simplefilter("ignore")

xr.set_options(display_style='html');

Using cf-xarray in version 0.6.2.dev2+g1fde526
Using xESMF in version 0.6.1
Using clisops in version 0.6.5


##### Initialize test data

In [2]:
# Initialize mini-esgf-data
MINIESGF_URL="https://github.com/roocs/mini-esgf-data"
branch = "master"
MINIESGF = Path(Path.home(),".mini-esgf-data", branch)

# Retrieve mini-esgf test data
if not os.path.isdir(MINIESGF):
    repo = Repo.clone_from(MINIESGF_URL, MINIESGF)
    repo.git.checkout(branch)
else:
    repo = Repo(MINIESGF)
    repo.git.checkout(branch)
    repo.remotes[0].pull()
    
MINIESGF=Path(MINIESGF,"test_data")

## 1 Default approach

#### Load the datasets

In [3]:
# Load the dataset
ds_path_o3 = Path(MINIESGF, "badc/cmip6/data/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/AERmon/"
                            "o3/gn/v20190710/o3_AERmon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_185001.nc")
ds_o3 = xr.open_dataset(ds_path_o3)
ds_o3

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 3, lon: 192, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * lev        (lev) float64 0.9961 0.9826 0.959
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lon_bnds   (lon, bnds) float64 ...
    lat_bnds   (lat, bnds) float64 ...
    lev_bnds   (lev, bnds) float64 ...
    ap         (lev) float64 ...
    b          (lev) float64 ...
    ap_bnds    (lev, bnds) float64 ...
    b_bnds     (lev, bnds) float64 ...
    ps         (time, lat, lon) float32 ...
    o3         (time, lev, lat, lon) float32 ...
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Tue Oct 26 11:48:46 2021: cdo sellevidx,1/3 -selt...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/52bee89e-e989-4005-8663-e60210d421d5
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [4]:
ds_path_tos = Path(MINIESGF, "badc/cmip6/data/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/Omon/"
                             "tos/gn/v20190710/tos_Omon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_185001-186912.nc")
ds_tos = xr.open_dataset(ds_path_tos)
ds_tos

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 6, j: 5, time: 240, vertices: 4)
Coordinates:
  * i                   (i) int32 0 45 90 135 180 225
  * j                   (j) int32 0 45 90 135 180
    latitude            (j, i) float64 ...
    longitude           (j, i) float64 ...
  * time                (time) datetime64[ns] 1850-01-16T12:00:00 ... 1869-12...
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) datetime64[ns] ...
    tos                 (time, j, i) float32 ...
    vertices_latitude   (j, i, vertices) float64 ...
    vertices_longitude  (j, i, vertices) float64 ...
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/40326101-1a73-46bb-a1d8-edb6f6a4da7c
    NCO:                    netCDF Operators version 4.7.8 (Homepage = http:/...

#### Calculate the regridding weights

In [5]:
# Specify a global 1 deg grid as target grid
ds_out = xe.util.grid_global(1,1)

In [6]:
# Create regridding weights
def regrid(ds_in, ds_out, method='nearest_s2d', locstream_in=False):
    """Convenience function for calculating regridding weights"""
    return xe.Regridder(ds_in, ds_out, method, locstream_in)

In [7]:
regridder_o3 = regrid(ds_o3, ds_out)
regridder_tos = regrid(ds_tos, ds_out)

#### Perform remapping

Important vertical coordinate variables are lost in the regridded dataset `ds_o3_g1`!
While the old bounds are kept in the regridded dataset `ds_tos_g1`.

In [8]:
ds_o3_g1 = regridder_o3(ds_o3, keep_attrs=True)
ds_tos_g1 = regridder_tos(ds_tos, keep_attrs=True)

In [9]:
ds_o3_g1

<xarray.Dataset>
Dimensions:  (lev: 3, time: 1, x: 360, y: 180)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-16T12:00:00
  * lev      (lev) float64 0.9961 0.9826 0.959
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: x, y
Data variables:
    ps       (time, y, x) float32 6.688e+04 6.695e+04 ... 1.013e+05 1.013e+05
    o3       (time, lev, y, x) float32 1.529e-08 1.529e-08 ... 2.34e-08 2.34e-08
Attributes: (12/50)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Tue Oct 26 11:48:46 2021: cdo sellevidx,1/3 -selt...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/52bee89e-e989-4005-8663-e60210d421d5
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...
    regrid_method:          nearest_s2d

In [10]:
ds_tos_g1

<xarray.Dataset>
Dimensions:             (time: 240, vertices: 4, x: 360, y: 180)
Coordinates:
  * time                (time) datetime64[ns] 1850-01-16T12:00:00 ... 1869-12...
    lon                 (y, x) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
    lat                 (y, x) float64 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5
Dimensions without coordinates: vertices, x, y
Data variables:
    tos                 (time, y, x) float32 2.37 2.37 2.37 ... -1.899 -1.899
    vertices_latitude   (vertices, y, x) float64 -63.83 -63.83 ... 83.34 83.34
    vertices_longitude  (vertices, y, x) float64 222.4 222.4 ... 272.1 272.1
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/40326101-1a73-46bb-a1d8-edb6f6a4da7c
    NCO:                    netCDF Operators version 4.7.8 (Homepage = http:/...
    regrid_method:          nearest_s2d

### What can be done?

* Get all coordinate variables and auxiliary coordinate variables to be recognized as coordinates by xarray.
  * by using the option decode_coordinates when loading the dataset
  * by making use the clisops Grid methods (that could be implemented into xesmf itself)
* Store the remapped variables in ds_out and transfer all necessary (eg. non-horizontal) coordinate variables to ds_out

## 2 Approach using `xarray.open_dataset(path_to_ds, decode_coords='all')`

That causes the variable `ps` in `ds_o3` to be defined under `xarray.Dataset.coords`, leading to xESMF dropping this variable when calling `xesmf.Regridder.__call__`.
For `ds_tos` the `cf_xarray.CFAccessor` cannot uniquely identify the horizontal coordinates until incl. v0.6.1 (https://github.com/xarray-contrib/cf-xarray/pull/264). Here, the old vertices are not dropped, but the new bounds are not assigned to the dataset.

In [11]:
# Load datasets with decode_coords="all" parameter
# for ds_o3 it will cause ps to be identified as coordinate (and therefore dropped during remapping)
ds_o3 = xr.open_dataset(ds_path_o3, decode_coords='all')
ds_o3

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 3, lon: 192, time: 1)
Coordinates: (12/13)
  * time       (time) datetime64[ns] 1850-01-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] ...
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 ...
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 ...
    ...         ...
    lev_bnds   (lev, bnds) float64 ...
    ap         (lev) float64 ...
    b          (lev) float64 ...
    ap_bnds    (lev, bnds) float64 ...
    b_bnds     (lev, bnds) float64 ...
    ps         (time, lat, lon) float32 ...
Dimensions without coordinates: bnds
Data variables:
    o3         (time, lev, lat, lon) float32 ...
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Tue Oct 26 11:48:46 2021: cdo sellevidx,1/3 -selt...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/52bee89e-e989-4005-8663-e60210d421d5
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [12]:
ds_tos = xr.open_dataset(ds_path_tos, decode_coords='all')
ds_tos

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 6, j: 5, time: 240, vertices: 4)
Coordinates:
  * i                   (i) int32 0 45 90 135 180 225
  * j                   (j) int32 0 45 90 135 180
    latitude            (j, i) float64 ...
    longitude           (j, i) float64 ...
  * time                (time) datetime64[ns] 1850-01-16T12:00:00 ... 1869-12...
    time_bnds           (time, bnds) datetime64[ns] ...
    vertices_latitude   (j, i, vertices) float64 ...
    vertices_longitude  (j, i, vertices) float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    tos                 (time, j, i) float32 ...
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/40326101-1a73-46bb-a1d8-edb6f6a4da7c
    NCO:                    netCDF Operators version 4.7.8 (Homepage = http:/...

In [13]:
regridder_o3 = regrid(ds_o3, ds_out)
regridder_tos = regrid(ds_tos, ds_out)

#### All coordinates are kept, but the variable ps is dropped. However, it should have been remapped as well.

In [14]:
ds_o3_g1 = regridder_o3(ds_o3, keep_attrs=True)
ds_o3_g1

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 3, time: 1, x: 360, y: 180)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 1850-01-01 1850-02-01
  * lev        (lev) float64 0.9961 0.9826 0.959
    lev_bnds   (lev, bnds) float64 1.0 0.9923 0.9923 0.973 0.973 0.9449
    ap         (lev) float64 0.0 0.0 36.03
    b          (lev) float64 0.9961 0.9826 0.9586
    ap_bnds    (lev, bnds) float64 0.0 0.0 0.0 0.0 0.0 72.06
    b_bnds     (lev, bnds) float64 1.0 0.9923 0.9923 0.973 0.973 0.9442
    lon        (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat        (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: bnds, x, y
Data variables:
    o3         (time, lev, y, x) float32 1.529e-08 1.529e-08 ... 2.34e-08
Attributes: (12/50)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Tue Oct 26 11:48:46 2021: cdo sellevidx,1/3 -selt...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/52bee89e-e989-4005-8663-e60210d421d5
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...
    regrid_method:          nearest_s2d

#### The old vertices are dropped, but the new bounds are not assigned to the dataset

In [15]:
ds_tos_g1 = regridder_tos(ds_tos, keep_attrs=True)
ds_tos_g1

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 240, x: 360, y: 180)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 1869-12-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 1850-01-01 1850-02-01 ... 1870-01-01
    lon        (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat        (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: bnds, x, y
Data variables:
    tos        (time, y, x) float32 2.37 2.37 2.37 2.37 ... -1.899 -1.899 -1.899
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/40326101-1a73-46bb-a1d8-edb6f6a4da7c
    NCO:                    netCDF Operators version 4.7.8 (Homepage = http:/...
    regrid_method:          nearest_s2d

## 3 Approach using clisops Grid methods to (re)set `xarray.Dataset.data_vars` and `xarray.Dataset.coords` and storing remapped data in ds_out

Variable `ps` in `ds_o3` is not remapped as well, all necessary coordinate variables are kept.
For `ds_tos` the horizontal bounds are dropped.

#### Build Grid objects

In [16]:
grid_o3 = Grid(ds = ds_o3)
grid_o3.ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lev: 3, lon: 192, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 1850-01-01 1850-02-01
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon_bnds   (lon, bnds) float64 -0.9375 0.9375 0.9375 ... 357.2 357.2 359.1
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    lat_bnds   (lat, bnds) float64 -89.5 -87.65 -87.65 ... 87.65 87.65 89.5
  * lev        (lev) float64 0.9961 0.9826 0.959
    lev_bnds   (lev, bnds) float64 1.0 0.9923 0.9923 0.973 0.973 0.9449
    ap         (lev) float64 0.0 0.0 36.03
    b          (lev) float64 0.9961 0.9826 0.9586
    ap_bnds    (lev, bnds) float64 0.0 0.0 0.0 0.0 0.0 72.06
    b_bnds     (lev, bnds) float64 1.0 0.9923 0.9923 0.973 0.973 0.9442
Dimensions without coordinates: bnds
Data variables:
    ps         (time, lat, lon) float32 7.05e+04 7.044e+04 ... 1.009e+05
    o3         (time, lev, lat, lon) float32 1.68e-08 1.679e-08 ... 2.32e-08
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Tue Oct 26 11:48:46 2021: cdo sellevidx,1/3 -selt...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/52bee89e-e989-4005-8663-e60210d421d5
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [17]:
grid_tos = Grid(ds = ds_tos)
grid_tos.ds

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 6, j: 5, time: 240, vertices: 4)
Coordinates:
  * i                   (i) int32 0 45 90 135 180 225
  * j                   (j) int32 0 45 90 135 180
    latitude            (j, i) float64 76.36 71.8 67.83 ... -57.62 -53.35 -54.41
    longitude           (j, i) float64 312.7 304.7 313.5 ... 359.8 53.79 110.0
  * time                (time) datetime64[ns] 1850-01-16T12:00:00 ... 1869-12...
    time_bnds           (time, bnds) datetime64[ns] 1850-01-01 ... 1870-01-01
    vertices_latitude   (j, i, vertices) float64 76.38 76.49 ... -54.84 -54.06
    vertices_longitude  (j, i, vertices) float64 313.5 313.3 ... 110.6 110.8
Dimensions without coordinates: bnds, vertices
Data variables:
    tos                 (time, j, i) float32 nan nan nan ... 1.699 4.365 4.734
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/40326101-1a73-46bb-a1d8-edb6f6a4da7c
    NCO:                    netCDF Operators version 4.7.8 (Homepage = http:/...

In [18]:
grid_out_o3 = Grid(ds = ds_out)
grid_out_tos = Grid(ds = ds_out)
grid_out_o3.ds

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 180, lon: 360)
Coordinates:
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat_bnds  (lat, bnds) float64 -90.0 -89.0 -89.0 -88.0 ... 89.0 89.0 90.0
    lon_bnds  (lon, bnds) float64 -180.0 -179.0 -179.0 ... 179.0 179.0 180.0
Dimensions without coordinates: bnds
Data variables:
    *empty*

#### Manually transfer coordinates and attributes

`xarray.Dataset.attrs` and essential `xarray.Dataset.coords` have to be moved manually.

In [19]:
grid_out_o3._transfer_coords(grid_o3)
grid_out_tos._transfer_coords(grid_tos)

#### Build regridder and regrid

In [20]:
regridder_o3 = regrid(grid_o3.ds, grid_out_o3.ds)
regridder_tos = regrid(grid_tos.ds, grid_out_tos.ds)

In [21]:
grid_out_o3.ds["o3"] = regridder_o3(grid_o3.ds["o3"])
grid_out_o3.ds["ps"] = regridder_o3(grid_o3.ds["ps"])
grid_out_o3.ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lev: 3, lon: 360, time: 1)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat_bnds   (lat, bnds) float64 -90.0 -89.0 -89.0 -88.0 ... 89.0 89.0 90.0
    lon_bnds   (lon, bnds) float64 -180.0 -179.0 -179.0 ... 179.0 179.0 180.0
  * time       (time) datetime64[ns] 1850-01-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 1850-01-01 1850-02-01
    ap         (lev) float64 0.0 0.0 36.03
    b          (lev) float64 0.9961 0.9826 0.9586
  * lev        (lev) float64 0.9961 0.9826 0.959
    ap_bnds    (lev, bnds) float64 0.0 0.0 0.0 0.0 0.0 72.06
    b_bnds     (lev, bnds) float64 1.0 0.9923 0.9923 0.973 0.973 0.9442
    lev_bnds   (lev, bnds) float64 1.0 0.9923 0.9923 0.973 0.973 0.9449
Dimensions without coordinates: bnds
Data variables:
    o3         (time, lev, lat, lon) float32 1.529e-08 1.529e-08 ... 2.34e-08
    ps         (time, lat, lon) float32 6.688e+04 6.695e+04 ... 1.013e+05
Attributes: (12/49)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    history:                Tue Oct 26 11:48:46 2021: cdo sellevidx,1/3 -selt...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variable_id:            o3
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/52bee89e-e989-4005-8663-e60210d421d5
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [22]:
grid_out_tos.ds["tos"] = regridder_tos(grid_tos.ds["tos"])
grid_out_tos.ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 240)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat_bnds   (lat, bnds) float64 -90.0 -89.0 -89.0 -88.0 ... 89.0 89.0 90.0
    lon_bnds   (lon, bnds) float64 -180.0 -179.0 -179.0 ... 179.0 179.0 180.0
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 1869-12-16T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 1850-01-01 1850-02-01 ... 1870-01-01
Dimensions without coordinates: bnds
Data variables:
    tos        (time, lat, lon) float32 2.37 2.37 2.37 ... -1.899 -1.899 -1.899
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/40326101-1a73-46bb-a1d8-edb6f6a4da7c
    NCO:                    netCDF Operators version 4.7.8 (Homepage = http:/...